<a href="https://colab.research.google.com/github/JoanChege/CHATBOT1/blob/main/chatbottrial6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.DATA PREPARATION

In [1]:
#first we need to import the required packages
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import tensorflow as tf
#from tensorflow.keras.preprocessing.text import Tokenizer

import numpy as np
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [11]:
#we load the json file and extract the required data.
#with open('/content/dataset.json') as file:
   # data = json.load(file)

words=[] #word list
#empty list
classes = []
documents = []
ignore_words = ['?', '!', ' ']

data = open('/content/dataset.json').read()
intents  = json.loads(data)

In [12]:
#create a dataframr from the json file
import pandas as pd
df = pd.json_normalize(intents)
df

,intents
0,"[{'tag': 'greeting', 'patterns': ['Hi', 'Hey',..."


Iterates over each intent in the 'intents' list.
For each intent, it iterates over the 'patterns' list associated with that intent.
It tokenizes each pattern (sentence) using the nltk library's 'word_tokenize' function and converts the sentence into a list of words.
It then extends the 'words' list with these newly tokenized words.
It creates a tuple (wordList, intent['tag']) and appends it to the 'documents' list.
It also checks if the tag of the current intent is already in the 'classes' list. If it's not, it appends the tag to the 'classes' list.

In [14]:
for intent in intents['intents']:
  for pattern in intent['patterns']:
    #we take each word and tokenize it
    wordlist = nltk.word_tokenize(pattern)
    words.extend(wordlist)

    #add documents
    documents.append((wordlist, intent['tag']))

    #add to words list
    for token in wordlist:
      words.append(token)

    #we add classes to our class list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

the first line takes each word in the words list and reduces the word to its base form by lemmatizing the word.
the second line sorts the words list and removes and duplicates. the sorting is in ascending order.
the next line sorts the classes list and removes any duplicate classes.
the pickle dump saves the words and classes to a file that is the words.pkl and classes.pkl and wb is the parameter that indicates that the file is bwing opened for writing in binary mode.
the above steps are used to convert the words into numerical form.

In [15]:
import collections

# assuming you have a list of words called 'word_list'
word_counts = collections.Counter(words)
num_words = len(word_counts)

print(f"The number of unique words in the vocabulary is {num_words}")

The number of unique words in the vocabulary is 686


In [16]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_words]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [ ]:
print(data)

In [17]:
#from nltk.tokenize import word_tokenize

#intialize the training data
training = []
output_empty = [0] * len(classes)
X_train = []
Y_train = []

for doc in documents:
  #intialize bag of words and generate for each document
  bow = []
  #words = word_tokenize(doc[0])
  #list of the tokenized words for the pattern
  pattern_words = doc[0]
  #lemmatize each word
  pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
  #create the bag of words array with , if the word match found in current pattern
  for wordlist in words:
    bow.append(1) if wordlist in pattern_words else bow.append(0)

  #output is 0 for each tag, and 1 for current tag
  #this create the output vector
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1


  #add the bag of words and output row to the training data
  #num_samples = 100
  #for i in range(num_samples):
  #training.append([bow,  output_row])
  #training = np.column_stack((training,[bow, output_row])) if len(training) else np.array([bow, output_row])
  if len(training):
    new_training = [[bow, output_row]]
    for i in range(len(training[0])):
      new_training.append(training[:, i])
  else:
    new_training = [[bow, output_row]]

  training = np.column_stack(new_training)
  #shuffle the features and turn into np.array
  random.shuffle(new_training)
  training = np.array(training, dtype=object) #could add ,dtype=object

  # Convert the training data to a list from a numpy array
  #training = training.tolist()
X_train = list(training[:,0].reshape(-1,1))
Y_train = list(training[:,1].reshape(-1,1))
print("The training data has been created")



/usr/local/lib/python3.10/dist-packages/numpy/lib/shape_base.py:652: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(v)


The training data has been created


In [ ]:
print(data)

Split the data into traing and testing sets

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.1)

Modelling using LSTM- LONG SHORT TERM MEMORY
ReLU- RECTIFIED LINEAR UNIT

In [19]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Bidirectional, Dropout, Dense

The model is a Sequential model, meaning that it consists of a sequence of layers, where the output of each layer is the input to the next layer.

---
Embedding layer, which converts a sequence of words into a sequence of dense vectors of fixed size. In this case, the Embedding layer has an input dimension of 384, an output dimension of 100, and the input length is set to the length of the data sentences.


---
The Bidirectional layer is a wrapper around the LSTM layer that allows the model to process the input sequence in both directions, left-to-right and right-to-left. This can help capture the context of the data.

---
Following the Bidirectional layer, there are two Dropout layers, which are used to prevent overfitting by randomly setting a fraction of the input units to 0 at each update during training time. The Dropout rate is set to 0.5.


---


Next, the Dense layer is added, which is a fully connected layer that is used to learn the classification of the data. The output dimension of the Dense layer is set to 64, and the activation function is set to 'relu'.


---

After the Dense layer, there is another Dropout layer, and finally, the Dense layer with a softmax activation function is added, which represents the final output of the model, where each element of the output vector represents the probability of the corresponding sentiment (positive or negative).




In [20]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.utils import to_categorical


In [21]:
num_words = 686


tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(X_train)

# Tokenizing sentences
X_train_tokenized = tokenizer.texts_to_sequences(X_train)

# Padding sequences to the same length
X_train_padded = pad_sequences(X_train_tokenized, maxlen=len(X_train[0]), padding='post')

# One-hot encoding the binary labels
Y_train_one_hot = to_categorical(Y_train)


AttributeError: ignored

In [22]:
model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=100, input_length=len(X_train[0])))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(Y_train[0]), activation='softmax'))


1.   Import the necessary modules.
2.Create a Sequential model.
3.Add an Embedding layer to the model, which will convert the words into dense vectors of fixed size.
4.Add a Bidirectional wrapper layer to the model. This layer is used to apply the LSTM in both forward and backward directions.
5.Add an LSTM layer to the model. In this case, it is used as the underlying layer within the Bidirectional wrapper.
6.Add a Dropout layer to the model to help prevent overfitting.
7.Add a Dense layer to the model, which is used to learn the classification of the movie review.
8.Add another Dropout layer to the model.
9.Add a Dense layer with a softmax activation function to the model, which represents the final output of the model, where each element of the output vector represents the probability of the corresponding sentiment (positive or negative).





In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 100)            68600     
                                                                 
 bidirectional (Bidirection  (None, 1, 200)            160800    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 1, 200)            0         
                                                                 
 dense (Dense)               (None, 1, 64)             12864     
                                                                 
 dropout_1 (Dropout)         (None, 1, 64)             0         
                                                                 
 dense_1 (Dense)             (None, 1, 1)              65        
                                                        

1.   The first layer is an embedding layer. This layer is responsible for converting words into fixed-size vectors.
2.  The second layer is a bidirectional LSTM layer. This layer takes the fixed-size vectors as input and processes them. The bidirectional part means that the LSTM layer learns the sequence from both the beginning and the end.
3.   The third layer is a dropout layer. This layer helps to prevent overfitting by randomly dropping out a percentage of neurons during training.
4.   The fourth layer is a dense layer. This layer combines the information from the LSTM layer to make a final prediction.
5.  The fifth layer is another dropout layer.
6.  The final layer is another dense layer. This layer has only one neuron and outputs the final probability that the given text is positive or negative.

---


To train the model, we pass it an input tensor and a target tensor. The input tensor contains the text, and the target tensor contains the labels. The model then learns to map the input text to the correct output label by adjusting its weights based on the loss function.


In [24]:
from keras.optimizers import SGD
#use Stochastic Gradient Descent optimizer to train the LSTM model
model.compile(optimizer = SGD(learning_rate=0.01,
                            momentum=0.9, #help to improve the training speed and model stability
                            nesterov=True),#If this argument is set to True, the momentum term will be computed in the direction of the previous step, not in the direction of the current gradient. This modification provides better convergence and improved performance.
              loss = 'sparse_categorical_crossentropy,',
              metrics = ['accuracy'])


1.   Lose-loss function that the model aims to minimize during training. In this case, we are using 'sparse_categorical_crossentropy', which is a loss function commonly used for multi-class classification problems.
2.   Optimizer- minimize the loss function during training
1.   metrics- it specifies the evaluation metrics to be used during training and evaluation. In this case, we are using 'accuracy' as our evaluation metric.
2.   List item





In [ ]:
Y_train_reshaped = tf.reshape(Y_train_one_hot, (len(Y_train_one_hot), -1, 2))

In [27]:
hist = model.fit(X_train, Y_train, epochs=10, batch_size=5, verbose=1)

ValueError: ignored